# SageMaker/DeepAR demo on electricity dataset

본 노트북은 다음 예제를 한글로 번역하고 일부 오류를 수정 반영하였습니다. [DeepAR electricity notebook](https://github.com/awslabs/amazon-sagemaker-examples/tree/master/introduction_to_amazon_algorithms/deepar_electricity).
<br>본 노트북은 다음 예제를 보완합니다. [DeepAR introduction notebook](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_amazon_algorithms/deepar_synthetic/deepar_synthetic.ipynb). 

본 예제에서 370명의 고객에 대한 시간별 에너지 사용량을 예측하는 유즈케이스를 SageMaker의 DeepAR로 어떻게 해결하는지 살펴볼 것입니다.  
사용할 데이터셋에 대한 자세한 내용은 다음 링크와 [dataset](https://archive.ics.uci.edu/ml/datasets/ElectricityLoadDiagrams20112014) 다음 논문들을 참고합니다. [[1](https://media.nips.cc/nipsbooks/nipspapers/paper_files/nips29/reviews/526.html)] and [[2](https://arxiv.org/abs/1704.04110)].  

본 예제를 통해 살펴볼 내용은 다음과 같습니다.
* 데이터셋 준비하기
* SageMaker Python SDK를 이용하여 DeepAR 모델을 학습하고 배포하기 
* 배포된 모델에 예측을 위한 요청 실행하기 
* DeepAR의 고급기능 살펴보기 : 결측치 처리, time feature 추가, 비정기 주기(frequency)와 카테고리 정보 사용하기

학습시간은 ml.c4.xlarge에서 실행시 약 40분 정도가 걸립니다. 추론은 ml.m4.xlarge에서 실행합니다. (이후 endpoint를 더이상 사용하지 않는 경우 endpoint를 삭제하십시오.)

보다 자세한 내용은 다음을 참고하십시오. [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar.html) 또는 [paper](https://arxiv.org/abs/1704.04110), 

In [ ]:
%matplotlib inline

import sys

from urllib.request import urlretrieve
import zipfile
from dateutil.parser import parse
import json
from random import shuffle
import random
import datetime
import os

import boto3
import s3fs
import sagemaker
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import IntSlider, FloatSlider, Checkbox

In [ ]:
# set random seeds for reproducibility
np.random.seed(42)
random.seed(42)

In [ ]:
sagemaker_session = sagemaker.Session()

필요시 다음 값을 업데이트하십시오. (옵션)
- S3 버킷과 prefix 이름 - 학습데이터와 모델데이터가 저장됩니다. S3의 리전은 본 노트북을 실행하는 리전과 동일해야 합니다. 
- IAM role arn - 학습과 호스팅에서 사용할 IAM 역할 (열할생성은 AWS IAM문서를 참고하십시오.)

In [ ]:
s3_bucket = sagemaker.Session().default_bucket()  # replace with an existing bucket if needed
s3_prefix = 'deepar-electricity-demo-notebook'    # prefix used for all data stored within the bucket

role = sagemaker.get_execution_role()             # IAM role to use by SageMaker

In [ ]:
region = sagemaker_session.boto_region_name

s3_data_path = "s3://{}/{}/data".format(s3_bucket, s3_prefix)
s3_output_path = "s3://{}/{}/output".format(s3_bucket, s3_prefix)

이제 학습을 실행할 컨테이너를 구성합니다.

In [ ]:
image_name = sagemaker.amazon.amazon_estimator.get_image_uri(region, "forecasting-deepar", "latest")

### 에너지사용 데이터 import 및 s3에 업로드하기 

UCI 데이터셋 리포지로부터 원본 데이터셋을 다운로드합니다. 

In [ ]:
DATA_HOST = "https://archive.ics.uci.edu"
DATA_PATH = "/ml/machine-learning-databases/00321/"
ARCHIVE_NAME = "LD2011_2014.txt.zip"
FILE_NAME = ARCHIVE_NAME[:-4]

In [ ]:
def progress_report_hook(count, block_size, total_size):
    mb = int(count * block_size // 1e6)
    if count % 500 == 0:
        sys.stdout.write("\r{} MB downloaded".format(mb))
        sys.stdout.flush()

if not os.path.isfile(FILE_NAME):
    print("downloading dataset (258MB), can take a few minutes depending on your connection")
    urlretrieve(DATA_HOST + DATA_PATH + ARCHIVE_NAME, ARCHIVE_NAME, reporthook=progress_report_hook)

    print("\nextracting data archive")
    zip_ref = zipfile.ZipFile(ARCHIVE_NAME, 'r')
    zip_ref.extractall("./")
    zip_ref.close()
else:
    print("File found skipping download")

다음은 데이터셋을 Pandas time series로 로드하고 변환을 진행하겠습니다. Pandas 를 이용할 경우 index조정이나 리샘플링 등이 보다 용이합니다.   
원본데이터는 15분 간격으로 에너지사용이 기록되어 있지만 여기서는 2시간 간격으로 조정하겠습니다. 

In [ ]:
data = pd.read_csv(FILE_NAME, sep=";", index_col=0, parse_dates=True, decimal=',')
num_timeseries = data.shape[1]
data_kw = data.resample('2H').sum() / 8
timeseries = []
for i in range(num_timeseries):
    timeseries.append(np.trim_zeros(data_kw.iloc[:,i], trim='f'))

처음 10명의 사용자에 대하여 2014년부터 2주간의 에너지 사용량 time series를 그래프로 그려보겠습니다.

In [ ]:
fig, axs = plt.subplots(5, 2, figsize=(20, 20), sharex=True)
axx = axs.ravel()
for i in range(0, 10):
    timeseries[i].loc["2014-01-01":"2014-01-14"].plot(ax=axx[i])
    axx[i].set_xlabel("date")    
    axx[i].set_ylabel("kW consumption")   
    axx[i].grid(which='minor', axis='x')

### 학습, 검증데이터셋 분리

종종 모델을 평가하고 하이퍼파라미터 튜닝을 실행하기 위해 별도의 데이터셋을 이용한 에러 평가 매트릭이 필요합니다. 여기서는 가용한 데이터셋을 학습과 테스트용 셋으로 나누겠습니다. 분류나 회귀와 같은 일반적인 머신러닝 작업에서는 주로 랜덤하게 샘플을 추출하여 학습과 테스트셋을 생성하지만, 시계열 예측과 같은 문제에서는 시간순서를 기준으로 학습과 테스트셋을 나누어야 합니다. 

본 예제에서는 시계열의 마지막 부분을 평가용도로 분리하고 시계열의 앞부분을 학습용으로 사용하겠습니다. 

In [ ]:
# we use 2 hour frequency for the time series
freq = datetime.timedelta(hours=2)

# we predict for 7 days
prediction_length = 7 * 12

# we also use 7 days as context length, this is the number of state updates accomplished before making predictions
context_length = 7 * 12

2014-01-01 부터 2014-09-01 까지의 데이터를 학습에 사용할 것입니다. 

In [ ]:
start_dataset = pd.Timestamp("2014-01-01 00:00:00", freq=freq)
end_training = pd.Timestamp("2014-09-01 00:00:00", freq=freq)

DeepAR 알고리즘에서는 JSON 입력포맷을 요구합니다. JSON 오브젝트 형태로 time series를 표현해야 합니다. 가장 간단한 방법은 각 시계열을 ``start``에 해당하는 시계열 시작점과 ``target``에 해당하는 시계열값의 리스트형태로 구성하는 것입니다. 

보다 복잡한 케이스로 ``dynamic_feat`` 항목을 이용하여 다른 시계열값을 feature로 입력받을수 있고, ``cat`` 항목을 이용하여 명목형 feature를 입력받을 수 있습니다. 이부분은 본 예제의 후반부에서 다루어집니다. 


In [ ]:
training_data = [
    {
        "start": str(start_dataset),
        "target": ts[start_dataset:end_training - 1*freq].tolist()  # We use -1, because pandas indexing includes the upper bound 
    }
    for ts in timeseries
]
print(len(training_data))

학습용 데이터의 이후 기간 데이터를 테스트용 데이터로 사용합니다. 이 데이터는 학습셋으로부터 이후 7일의 예측을 실행한 후 예측값과 실제값을 비교하여 테스트 스코어를 계산하는데 사용됩니다.  
1주일 이상 기간에 대한 모델의 성능을 측정하기 위해 우리는 테스트 데이터셋을 학습데이터 범위 이후 1, 2, 3, 4주까지 확장하겠습니다. 이런식으로 우리는 모델에 대한 *rolling evaluation*을 실행하게 됩니다.

In [ ]:
num_test_windows = 4

test_data = [
    {
        "start": str(start_dataset),
        "target": ts[start_dataset:end_training + k * prediction_length * freq].tolist()
    }
    for k in range(1, num_test_windows + 1) 
    for ts in timeseries
]
print(len(test_data))

이제 dictionary를 `jsonlines`형태로 저장합니다. (DeepAR에서는 zip으로 압축된 jsonlines와 parquet포맷을 지원합니다.)

In [ ]:
def write_dicts_to_file(path, data):
    with open(path, 'wb') as fp:
        for d in data:
            fp.write(json.dumps(d).encode("utf-8"))
            fp.write("\n".encode('utf-8'))

In [ ]:
%%time
write_dicts_to_file("train.json", training_data)
write_dicts_to_file("test.json", test_data)

이제 우리는 로컬환경에 데이터파일을 가지고 있습니다. 이 데이터셋을 S3로 복사합니다. 네트워크환경에 따라 이 부분은 수분 정도가 소요됩니다. 

In [ ]:
s3 = boto3.resource('s3')
def copy_to_s3(local_file, s3_path, override=False):
    assert s3_path.startswith('s3://')
    split = s3_path.split('/')
    bucket = split[2]
    path = '/'.join(split[3:])
    buk = s3.Bucket(bucket)
    
    if len(list(buk.objects.filter(Prefix=path))) > 0:
        if not override:
            print('File s3://{}/{} already exists.\nSet override to upload anyway.\n'.format(s3_bucket, s3_path))
            return
        else:
            print('Overwriting existing file')
    with open(local_file, 'rb') as data:
        print('Uploading file to {}'.format(s3_path))
        buk.put_object(Key=path, Body=data)

In [ ]:
%%time
copy_to_s3("train.json", s3_data_path + "/train/train.json")
copy_to_s3("test.json", s3_data_path + "/test/test.json")

S3에 복사한 데이터를 확인합니다.

In [ ]:
s3filesystem = s3fs.S3FileSystem()
with s3filesystem.open(s3_data_path + "/train/train.json", 'rb') as fp:
    print(fp.readline().decode("utf-8")[:100] + "...")

여기까지 학습을 위한 데이터셋 작업을 마무리하였습니다. 다음은 DeepAR을 이용하여 학습과 모델 생성 및 예측을 진행합니다.

### 모델 학습

이제 학습작업을 실행하기 위한 Estimator를 선언합니다.

In [ ]:
estimator = sagemaker.estimator.Estimator(
    sagemaker_session=sagemaker_session,
    image_name=image_name,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.c4.2xlarge',
    base_job_name='deepar-electricity-demo',
    output_path=s3_output_path
)

다음은 학습을 위한 하이퍼파라미터를 정의합니다. 예를 들어, time series의 주기(frequency)가 지정되었고, 과거시점을 바라보는 데이터포인트 범위가 context_length를 통해 지정되었고, 예측을 실행할 기간이 prediction_length를 이용하여 지정되었습니다. 모델과 관련하여 레이어의 수와 레이어당 셀의 수, likelihood function 등을 지정할 수 있으며 epoch수 batch size, learning rate 등과 같은 학습옵션을 지정할 수 있습니다.  
본 예제에서는 기본값을 사용하겠습니다. 하이퍼파리미터를 보다 세밀하게 튜닝하려면 [Sagemaker Automated Model Tuning](https://aws.amazon.com/blogs/aws/sagemaker-automatic-model-tuning/)을 이용할 수 있습니다. 


In [ ]:
hyperparameters = {
    "time_freq": '2H',
    "epochs": "400",
    "early_stopping_patience": "40",
    "mini_batch_size": "64",
    "learning_rate": "5E-4",
    "context_length": str(context_length),
    "prediction_length": str(prediction_length)
}

In [ ]:
estimator.set_hyperparameters(**hyperparameters)

이제 학습을 실행할 준비가 되었습니다. SAgeMaker는 EC2 인스턴스를 시작하고 S3를 다운로드하고, 학습된 진행시킨 후, 학습된 모델을 저장할 것입니다. 

만약 본 예제처럼 `test`데이터 채널을 입력한 경우, DeepAR 은 이 테스트셋을 이용하여 모델의 정확도(accuracy) 매트릭을 계산할 것입니다. 계산은 테스트셋의 각 timeseries의 마지막 `prediction_lengh`포인트를 예측하고 이를 실제값과 비교하는 방식으로 이루어집니다. 

**Note:** 다음 셀의 실행은 10분 이상 걸릴 수 있습니다. 실행시간은 데이터사이즈, 모델복잡도, 학습옵션 등에 따라 달라집니다. 

In [ ]:
%%time
data_channels = {
    "train": "{}/train/".format(s3_data_path),
    "test": "{}/test/".format(s3_data_path)
}

estimator.fit(inputs=data_channels, wait=True)

본 예제에서는 테스트셋을 지정하였기 때문에 이를 이용한 평가 매트릭이 계산되고 로깅되었습니다. (로그의 마지막을 보십시오.)  
매트릭에 대한 자세한 정보는 [our documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar.html)를 참고하십시오. 이 매트릭을 이용하여 [Automated Model Tuning service](https://aws.amazon.com/blogs/aws/sagemaker-automatic-model-tuning/)를 통해 파라미터를 최적화하고 모델을 튜닝할 수 있습니다. 

### Endpoint와 predictor 생성하기

이제 모델이 학습되었고, 이를 이용하여 예측을 수행할 수 있도록 엔드포인트(Endpoint)에 배포하겠습니다. 

**Note: 본 예제를 완료한 후 반드시 endpoint를 삭제하십시오. 삭제코드는 본 노트북의 맨 아래에 있습니다. 반드시 이 부분을 실행하십시오. 

Endpoint에 질의를 던지고 예측결과를 받기 위해, 다음 유틸리티 클래스를 정의합니다. 이 클래스는 JSON strings가 아닌 `pandas.Series`를 요청으로 사용할 수 있도록 합니다. (SDK 참조 : https://sagemaker.readthedocs.io/en/stable/api/inference/predictors.html?highlight=realtimepredictor#sagemaker.predictor.RealTimePredictor)

In [ ]:
class DeepARPredictor(sagemaker.predictor.RealTimePredictor):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, content_type=sagemaker.content_types.CONTENT_TYPE_JSON, **kwargs)
        
    def predict(self, ts, cat=None, dynamic_feat=None, 
                num_samples=100, return_samples=False, quantiles=["0.1", "0.5", "0.9"]):
        """Requests the prediction of for the time series listed in `ts`, each with the (optional)
        corresponding category listed in `cat`.
        
        ts -- `pandas.Series` object, the time series to predict
        cat -- integer, the group associated to the time series (default: None)
        num_samples -- integer, number of samples to compute at prediction time (default: 100)
        return_samples -- boolean indicating whether to include samples in the response (default: False)
        quantiles -- list of strings specifying the quantiles to compute (default: ["0.1", "0.5", "0.9"])
        
        Return value: list of `pandas.DataFrame` objects, each containing the predictions
        """
        prediction_time = ts.index[-1] + datetime.timedelta(hours=2)
        quantiles = [str(q) for q in quantiles]
        req = self.__encode_request(ts, cat, dynamic_feat, num_samples, return_samples, quantiles)
        res = super(DeepARPredictor, self).predict(req)
        return self.__decode_response(res, ts.index.freq, prediction_time, return_samples)
    
    def __encode_request(self, ts, cat, dynamic_feat, num_samples, return_samples, quantiles):
        instance = series_to_dict(ts, cat if cat is not None else None, dynamic_feat if dynamic_feat else None)

        configuration = {
            "num_samples": num_samples,
            "output_types": ["quantiles", "samples"] if return_samples else ["quantiles"],
            "quantiles": quantiles
        }
        
        http_request_data = {
            "instances": [instance],
            "configuration": configuration
        }
        
        return json.dumps(http_request_data).encode('utf-8')
    
    def __decode_response(self, response, freq, prediction_time, return_samples):
        # we only sent one time series so we only receive one in return
        # however, if possible one will pass multiple time series as predictions will then be faster
        predictions = json.loads(response.decode('utf-8'))['predictions'][0]
        prediction_length = len(next(iter(predictions['quantiles'].values())))
#         prediction_index = pd.DatetimeIndex(start=prediction_time, freq=freq, periods=prediction_length)       
#         print(prediction_time)
#         print(type(prediction_time))
#         print(prediction_length)
#         print(type(prediction_length))
#         print(freq)
#         print(type(freq))
        
        prediction_index = pd.date_range(prediction_time, prediction_time + freq * (prediction_length-1), freq=freq)
#         print(prediction_index)
        
        if return_samples:
            dict_of_samples = {'sample_' + str(i): s for i, s in enumerate(predictions['samples'])}
        else:
            dict_of_samples = {}
        return pd.DataFrame(data={**predictions['quantiles'], **dict_of_samples}, index=prediction_index)

    def set_frequency(self, freq):
        self.freq = freq
        
def encode_target(ts):
    return [x if np.isfinite(x) else "NaN" for x in ts]        

def series_to_dict(ts, cat=None, dynamic_feat=None):
    """Given a pandas.Series object, returns a dictionary encoding the time series.

    ts -- a pands.Series object with the target time series
    cat -- an integer indicating the time series category

    Return value: a dictionary
    """
    obj = {"start": str(ts.index[0]), "target": encode_target(ts)}
    if cat is not None:
        obj["cat"] = cat
    if dynamic_feat is not None:
        obj["dynamic_feat"] = dynamic_feat        
    return obj

위에서 정의한 DeepARPredictor 클래스를 활용하여 모델을 배포하고 endpoint를 생성합니다. 

In [ ]:
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.m4.xlarge',
    predictor_cls=DeepARPredictor)

### 예측 및 결과그래프 작성 

이제 예측을 위해 `predictor`를 사용할 수 있습니다.

In [ ]:
predictor.predict(ts=timeseries[120], quantiles=[0.10, 0.5, 0.90]).head()

다음은 모델이 예측한 결과를 그래프로 표현하는 함수입니다. 

In [ ]:
def plot(
    predictor, 
    target_ts, 
    cat=None, 
    dynamic_feat=None, 
    forecast_date=end_training, 
    show_samples=False, 
    plot_history=7 * 12,
    confidence=80
):
    print("calling served model to generate predictions starting from {}".format(str(forecast_date)))
    assert(confidence > 50 and confidence < 100)
    low_quantile = 0.5 - confidence * 0.005
    up_quantile = confidence * 0.005 + 0.5
        
    # we first construct the argument to call our model
    args = {
        "ts": target_ts[:forecast_date],
        "return_samples": show_samples,
        "quantiles": [low_quantile, 0.5, up_quantile],
        "num_samples": 100
    }


    if dynamic_feat is not None:
        args["dynamic_feat"] = dynamic_feat
        fig = plt.figure(figsize=(20, 6))
        ax = plt.subplot(2, 1, 1)
    else:
        fig = plt.figure(figsize=(20, 3))
        ax = plt.subplot(1,1,1)
    
    if cat is not None:
        args["cat"] = cat
        ax.text(0.9, 0.9, 'cat = {}'.format(cat), transform=ax.transAxes)

    # call the end point to get the prediction
    prediction = predictor.predict(**args)

    # plot the samples
    if show_samples: 
        for key in prediction.keys():
            if "sample" in key:
                prediction[key].plot(color='lightskyblue', alpha=0.2, label='_nolegend_')
                
                
    # plot the target
    target_section = target_ts[forecast_date-plot_history:forecast_date+prediction_length]
    target_section.plot(color="black", label='target')
    
    # plot the confidence interval and the median predicted
    ax.fill_between(
        prediction[str(low_quantile)].index, 
        prediction[str(low_quantile)].values, 
        prediction[str(up_quantile)].values, 
        color="b", alpha=0.3, label='{}% confidence interval'.format(confidence)
    )
    prediction["0.5"].plot(color="b", label='P50')
    ax.legend(loc=2)    
    
    # fix the scale as the samples may change it
    ax.set_ylim(target_section.min() * 0.5, target_section.max() * 1.5)
    
    if dynamic_feat is not None:
        for i, f in enumerate(dynamic_feat, start=1):
            ax = plt.subplot(len(dynamic_feat) * 2, 1, len(dynamic_feat) + i, sharex=ax)
            feat_ts = pd.Series(
                index=pd.DatetimeIndex(start=target_ts.index[0], freq=target_ts.index.freq, periods=len(f)),
                data=f
            )
            feat_ts[forecast_date-plot_history:forecast_date+prediction_length].plot(ax=ax, color='g')

방금 정의한 함수를 이용하여 임의의 고객에 대한 미래 특정시점의 예측값을 볼 수 있습니다. 

각 요청에 대한 예측값은 실시간으로 모델을 호출하여 얻습니다.

여기서 우리는 주말의 사무실의 전력소비를 예측합니다. 
보고자하는 시계열과 예측날짜를 선택하고 `Run Interact`를 클릭하여 엔드포인트를 호출한 결과로 리턴된 예측값을 그래프로 살펴봅니다. 

We can interact with the function previously defined, to look at the forecast of any customer at any point in (future) time. 

For each request, the predictions are obtained by calling our served model on the fly.

Here we forecast the consumption of an office after week-end (note the lower week-end consumption). 
You can select any time series and any forecast date, just click on `Run Interact` to generate the predictions from our served endpoint and see the plot.

In [ ]:
style = {'description_width': 'initial'}

In [ ]:
@interact_manual(
    customer_id=IntSlider(min=0, max=369, value=91, style=style), 
    forecast_day=IntSlider(min=0, max=100, value=51, style=style),
    confidence=IntSlider(min=60, max=95, value=80, step=5, style=style),
    history_weeks_plot=IntSlider(min=1, max=20, value=1, style=style),
    show_samples=Checkbox(value=False),
    continuous_update=False
)
def plot_interact(customer_id, forecast_day, confidence, history_weeks_plot, show_samples):
    plot(
        predictor,
        target_ts=timeseries[customer_id],
        forecast_date=end_training + datetime.timedelta(days=forecast_day),
        show_samples=show_samples,
        plot_history=history_weeks_plot * 12 * 7,
        confidence=confidence
    )

# 고급 기능

지금까지 간단한 예제를 통해 DeepAR을 위한 데이터셋을 준비하고 실행하는 방법을 살펴보았습니다. 

DeepAR에는 다음 기능들이 추가로 제공됩니다:

- 결측치 처리: DeepAR은 학습과 추론단계에서 모두 time series내의 결측치를 처리할 수 있습니다. 
- 추가 시계열 features: DeepAR은 hour of day 등 디폴트 time series를 추가로 제공합니다. 그리고 `dynamic_feat`를 이용하여 사용자 정의 시계열 feature도 입력할 수 있습니다. 
- 주기(frequency) 일반화 : 기본 주기(minutes `min`, hours `H`, days `D`, weeks `W`, month `M`)로부터 정수를 곱한 값을 지원합니다. (`15min`, `2H` 등)
- 명목(category) 변수: 만약 timeseries가 다른 그룹들(제품그룹, 타입, 지역, 등)에 속해 있다면 이 정보는 `cat`항목을 이용하여 추가로 명목형 변수로 입력될 수 있습니다.

바로 다음 예시를 통해 결측치 처리 기능을 확인하겠습니다. 이 부분은 앞서 진행한 에너지소비 데이터를 재사용하지만 기능의 소개를 위해 일부 인위적인 조작을 할 것입니다.
- 결측치 지원기능 확인을 위해 임의로 time series의 일부를 마스킹합니다. 
- '특별일'을 위한 또 다른 timeseries를 생성하고 이 날은 관측값이 높게 나오도록 조작합니다. 
- '특별일'을 나타태는 timeseries를 학습의 feature로 입력합니다. 


## 데이터셋 준비

앞서 언급한 바와 같이 '특별일'을 생성하고 해당날짜에 timeseries값을 높에 나오도록 조정해 보겠습니다. 이 시뮬레이션은 실제 상황에서는 특정 기간에 실행되면서 여러분의 timeseries에 영향을 미치는 프로모션이나 이벤트가 될 것입니다. 

In [ ]:
def create_special_day_feature(ts, fraction=0.05):
    # First select random day indices (plus the forecast day)
    num_days = (ts.index[-1] - ts.index[0]).days
    rand_indices = list(np.random.randint(0, num_days, int(num_days * 0.1))) + [num_days]
    
    feature_value = np.zeros_like(ts)
    for i in rand_indices:
        feature_value[i * 12: (i + 1) * 12] = 1.0
    feature = pd.Series(index=ts.index, data=feature_value)
    return feature

def drop_at_random(ts, drop_probability=0.1):
    assert(0 <= drop_probability < 1)
    random_mask = np.random.random(len(ts)) < drop_probability
    return ts.mask(random_mask)

In [ ]:
special_day_features = [create_special_day_feature(ts) for ts in timeseries]

In [ ]:
fig, axs = plt.subplots(5, 2, figsize=(20, 20), sharex=True)
axx = axs.ravel()
for i in range(0, 10):
    ax = axx[i]
    ts = time_series_processed[i][:400]
    ts.plot(ax=ax)
    ax.set_ylim(-0.1 * ts.max(), ts.max())
    ax2 = ax.twinx()
    special_day_features[i][:400].plot(ax=ax2, color='g')
    ax2.set_ylim(-0.2, 7)

'특별일'의 생성 다음에는 임의로 특정 시점의 값을 제거해 보겠습니다.  
아래 그림은 일부 샘플 timeseries를 보여주고 있습니다. 그림에서 초록색 그래프가 '특별일' 입니다.

In [ ]:
timeseries_uplift = [ts * (1.0 + feat) for ts, feat in zip(timeseries, special_day_features)]
time_series_processed = [drop_at_random(ts) for ts in timeseries_uplift]

In [ ]:
%%time

training_data_new_features = [
    {
        "start": str(start_dataset),
        "target": encode_target(ts[start_dataset:end_training]),
        "dynamic_feat": [special_day_features[i][start_dataset:end_training].tolist()]
    }
    for i, ts in enumerate(time_series_processed)
]
print(len(training_data_new_features))

# as in our previous example, we do a rolling evaluation over the next 7 days
num_test_windows = 7

test_data_new_features = [
    {
        "start": str(start_dataset),
        "target": encode_target(ts[start_dataset:end_training + freq*k*prediction_length]),
        "dynamic_feat": [special_day_features[i][start_dataset:end_training + freq*k*prediction_length].tolist()]
    }
#     freq = datetime.timedelta(hours=2)
    
    for k in range(1, num_test_windows + 1) 
    for i, ts in enumerate(timeseries_uplift)
]

In [ ]:
def check_dataset_consistency(train_dataset, test_dataset=None):
    d = train_dataset[0]
    has_dynamic_feat = 'dynamic_feat' in d
    if has_dynamic_feat:
        num_dynamic_feat = len(d['dynamic_feat'])
    has_cat = 'cat' in d
    if has_cat:
        num_cat = len(d['cat'])
    
    def check_ds(ds):
        for i, d in enumerate(ds):
            if has_dynamic_feat:
                assert 'dynamic_feat' in d
                assert num_dynamic_feat == len(d['dynamic_feat'])
                for f in d['dynamic_feat']:
                    assert len(d['target']) == len(f)
            if has_cat:
                assert 'cat' in d
                assert len(d['cat']) == num_cat
    check_ds(train_dataset)
    if test_dataset is not None:
        check_ds(test_dataset)
        
check_dataset_consistency(training_data_new_features, test_data_new_features)

In [ ]:
%%time
write_dicts_to_file("train_new_features.json", training_data_new_features)
write_dicts_to_file("test_new_features.json", test_data_new_features)

In [ ]:
%%time

s3_data_path_new_features = "s3://{}/{}-new-features/data".format(s3_bucket, s3_prefix)
s3_output_path_new_features = "s3://{}/{}-new-features/output".format(s3_bucket, s3_prefix)

print('Uploading to S3 this may take a few minutes depending on your connection.')
copy_to_s3("train_new_features.json", s3_data_path_new_features + "/train/train_new_features.json", override=True)
copy_to_s3("test_new_features.json", s3_data_path_new_features + "/test/test_new_features.json", override=True)

In [ ]:
%%time
estimator_new_features = sagemaker.estimator.Estimator(
    sagemaker_session=sagemaker_session,
    image_name=image_name,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.c4.2xlarge',
    base_job_name='deepar-electricity-demo-new-features',
    output_path=s3_output_path_new_features
)

hyperparameters = {
    "time_freq": '2H',
    "context_length": str(context_length),
    "prediction_length": str(prediction_length),
    "epochs": "400",
    "learning_rate": "5E-4",
    "mini_batch_size": "64",
    "early_stopping_patience": "40",
    "num_dynamic_feat": "auto",  # this will use the `dynamic_feat` field if it's present in the data
}
estimator_new_features.set_hyperparameters(**hyperparameters)

estimator_new_features.fit(
    inputs={
        "train": "{}/train/".format(s3_data_path_new_features),
        "test": "{}/test/".format(s3_data_path_new_features)
    }, 
    wait=True
)

처음 예제와 같이, 엔드포인트를 생성하고 실시간으로 예측값을 그래프로 그려 살펴봅니다. 

In [ ]:
%%time
predictor_new_features = estimator_new_features.deploy(
    initial_instance_count=1,
    instance_type='ml.m4.xlarge',
    predictor_cls=DeepARPredictor)

In [ ]:
customer_id = 120
predictor_new_features.predict(
    ts=time_series_processed[customer_id][:-prediction_length], 
    dynamic_feat=[special_day_features[customer_id].tolist()], 
    quantiles=[0.1, 0.5, 0.9]
).head()

처음 예제와 같이, 임의의 timeseries와 시점에 대하여 엔드포인트를 호출하고 예측값을 살펴봅니다. 

In [ ]:
@interact_manual(
    customer_id=IntSlider(min=0, max=369, value=13, style=style), 
    forecast_day=IntSlider(min=0, max=100, value=21, style=style),
    confidence=IntSlider(min=60, max=95, value=80, step=5, style=style),
    missing_ratio=FloatSlider(min=0.0, max=0.95, value=0.2, step=0.05, style=style),
    show_samples=Checkbox(value=False),
    continuous_update=False
)
def plot_interact(customer_id, forecast_day, confidence, missing_ratio, show_samples): 
    forecast_date = end_training + datetime.timedelta(days=forecast_day)
    target = time_series_processed[customer_id][start_dataset:forecast_date + prediction_length]
    target = drop_at_random(target, missing_ratio)
    dynamic_feat = [special_day_features[customer_id][start_dataset:forecast_date + prediction_length].tolist()]
    plot(
        predictor_new_features,
        target_ts=target, 
        dynamic_feat=dynamic_feat,
        forecast_date=forecast_date,
        show_samples=show_samples, 
        plot_history=7*12,
        confidence=confidence
    )

### Delete endpoints

In [ ]:
predictor.delete_endpoint()

In [ ]:
predictor_new_features.delete_endpoint()